# 智能RSI参数

根据个股一年来最低的RSI来判断当前股价是否处于最低位置

In [1]:
import traceback

await init_notebook()

def day(code, value):
    return round(rsiday.get_proba(code, value), 2)

def m30(code, value):        
    return round(rsi30.get_proba(code, value), 2)

In [2]:
# rsi = RsiStats(FrameType.MIN30)
# await rsi.calc("2021-09-30 15:00")

In [ ]:
# rsiday = RsiStats(FrameType.DAY)
# await rsiday.calc("2021-09-30")

In [2]:
async def check_rsi(names, tm=None):
    codes = [name_to_code(name) for name in names]
        
    results = []
    for code in codes:
        sec = Stock(code)
        name = sec.display_name
        
        ft = FrameType.MIN30
        end = arrow.get(tm) if tm else arrow.now()
        if end.hour == 0:
            end = tf.combine_time(end, hour = 15)
            
        start = tf.shift(tf.floor(end,ft), -39, ft)
        bars = await sec.load_bars(start, end, ft)
        bars = bars[np.isfinite(bars["close"])]
        
        r30 = relative_strength_index(bars["close"])[-3:]
        r30 = np.round(r30, 2)
        prsi30 = [rsi30.get_proba(code, r) for r in r30]
        
        ft = FrameType.DAY
        end = end.date()
        start = tf.day_shift(end, -39)
        bars  = await sec.load_bars(start, end, ft)
        bars = bars[np.isfinite(bars["close"])]
        
        rday = relative_strength_index(bars["close"])[-3:]
        rday = np.round(rday, 2)
        prday = [rsiday.get_proba(code, r) for r in rday]
        
        flag = ""
            
        try:
            if min(prsi30) < 0.1:
                print(f"{name}\t^\t{prsi30}\t30分钟")
            if min(prday) < 0.1:
                print(f"{name}\t^^\t{prday}\t日线")
                
            if max(prsi30) >= 0.9:
                print(f"{name}\tv\t{prsi30}\t30分钟")
            if max(prday) >= 0.9:
                print(f"{name}\tvv\t{prday}\t日线")

            if min(prsi30) < 0.1:
                flag += "^" 
            elif max(prsi30) >= 0.9:
                flag += "v"

            if min(prday) < 0.1:
                flag = " | ^^" 
            elif max(prday) >= 0.9:
                flag = " | vv"
        except Exception:
            continue
            
        results.append([name, code, prday[-1], prsi30[-1], flag])
        
    return pd.DataFrame(results, columns=["name", "code", "day", "30m", "Flag"])
            

# 机构调研股

In [ ]:

names = """
    英科医疗 泉峰汽车 晓鸣股份 宝通科技 合兴股份 新五丰 新亚电子 东南网架 味知香 日发精机 楚天科技 拓维信息 汤姆猫 拓邦股份 
    华中数控 朗特智能 国林科技 科博达 瑞丰光电 润建股份 泰晶科技 聚飞光电 四川双马 国民技术 东方电热 云图控股 恒星科技 苏州固锝
    金杯电工 伊戈尔 湘潭电化 鑫铂股份 运达股份 东方电气 联创电子
"""

df = await check_rsi(names)
df

# 持仓股和指数

In [3]:
names = [    
    "中国宝安",
    "方大集团",
    "天齐锂业",
    "利亚德",
    "东方电气",
    "上证指数"
]

await check_rsi(names)

2021-11-22 10:05:09,167 E 41896 omicron.client.quotes_fetcher:_quotes_server_get:29 | Cannot connect to host 127.0.0.1:3179 ssl:default [Connect call failed ('127.0.0.1', 3179)]
Traceback (most recent call last):
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packages/aiohttp/connector.py", line 969, in _wrap_create_connection
    return await self._loop.create_connection(*args, **kwargs)  # type: ignore  # noqa
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/asyncio/base_events.py", line 1056, in create_connection
    raise exceptions[0]
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/asyncio/base_events.py", line 1041, in create_connection
    sock = await self._connect_sock(
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/asyncio/base_events.py", line 955, in _connect_sock
    await self.sock_connect(sock, address)
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/asyncio/selector_events.py", line 502, in sock_conn

TypeError: object of type 'NoneType' has no len()

In [3]:
m30("000001.XSHG", 81.5)

0.94

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packages/cfg4py/core.py", line 357, in _load_from_local_file
    ext = _guess_extension()
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packages/cfg4py/core.py", line 305, in _guess_extension
    raise FileNotFoundError(msg)
FileNotFoundError: No config files present, or file format is not yaml.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packages/watchdog/observers/api.py", line 199, in run
    self.dispatch_events(self.event_queue, self.timeout)
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packages/watchdog/observers/api.py", line 372, in dispatch_events
    handler.dispa

In [ ]:
day("605005", 7.63)

# RSI底背离

2021年10月27日14：30，崇达技术（002815)出现30分钟底背离，同时日线RSI也处在低位（三个月前放量后，一直缩量阴跌）。此后连续放量，三个涨停。

In [ ]:
code = "002815.XSHE"
mbars = await get_bars(code, 60, '30m', end='2021-10-27 15:00')

mclose = mbars["close"]
mframe = mbars["frame"]

dbars = await get_bars(code, 90, '1d', end='2021-10-27 15:00')

In [ ]:
r30 = relative_strength_index(mclose)
divergency(r30[-20:], mclose[-20:])

In [ ]:
async def diagnose(code, frame_type):
    """提取个股的日线和30分钟线特征，并给出建议"""
    bars = await get_bars(code, 250, '1d')
    print(len(bars))
    close = bars["close"]

    features = {}
    
    features["price_change"] = {
        "max_year_gains": close[-1] / min(close) - 1 if len(close) >= 250 else np.nan,
        "max_year_loss": close[-1] / max(close) - 1 if len(close) >= 250 else np.nan,
        "max_quater_gains": close[-1] / min(close[-60:]) - 1 if len(close) >= 60 else np.nan,
        "max_quater_loss": close[-1] / max(close[-60:]) - 1 if len(close) >= 60 else np.nan,
        "max_mon_gains": close[-1] / min(close[-20:]) - 1 if len(close) >= 20 else np.nan,
        "max_mon_loss": close[-1] / max(close[-20:]) - 1 if len(close) >= 10 else np.nan,
        "max_week_gains": close[-1] / min(close[-5:]) - 1 if len(close) >= 5 else np.nan,
        "max_week_loss": close[-1] / max(close[-5:]) - 1 if len(close) >= 5 else np.nan
    }

    # 当前股价在一个季度以来的高度
    if len(bars) >= 60:
        features["altitude"] = altitude(bars[-60:])
    elif len(bars) >= 20:
        features["alt_20"] = altitude(bars[-20:])
        
    # 量能变化 over weeks
    volume = bars["volume"][-60:]
    
    weeks = len(volume) // 5
    wvol = np.sum(volume[:weeks * 5].reshape((-1, 5)), axis=1)
    
    months = len(volume) // 20
    mvol = np.sum(volume[:months * 20].reshape((-1, 20)), axis=1)
    
    features["vol_change"] = {
        "months": mvol[1:]/mvol[:-1] - 1,
        "weeks": wvol[1:]/wvol[:-1] - 1
    }
    
    # 反转特征
    stats = rsi30 if frame_type == FrameType.MIN30 else rsiday
    rsi = relative_strength_index(bars["close"], 6)
    features["rsi"] = rsi[-2:]
    features["prsi"] = [stats.get_proba(code, r) for r in rsi[-2:]]
    feattures["rsi_divergency"] = divergency(rsi, close)
    
    return features

In [ ]:
await diagnose("002139.XSHE")

In [ ]:
# 东方电热

Mbars = jq_get_bars("300217.XSHE", 90, '1d')
vol = Mbars["volume"]
weeks = len(vol) // 5
vol = np.sum(vol[:weeks*5].reshape((-1, 5)), axis=1)
plt.plot(vol[1:]/vol[:-1])

In [5]:
day("000153.XSHE", 31)

0.16